In [14]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path

# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Import the LogisticRegression module 
from sklearn.linear_model import LogisticRegression

# Import the model for the confusion matrix
from sklearn.metrics import confusion_matrix

# import the balanced accuracy score and the classification report
from sklearn.metrics import balanced_accuracy_score, classification_report

# Import the RandomOverSampler module from imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [15]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

lending_df = pd.read_csv(
    Path("Resources/lending_data.csv")
)

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [17]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
# The X variable should include all features except the target

X = lending_df.drop(columns=['loan_status'])

In [18]:
# Review the y variable Series

y[0:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [19]:
# Review the X variable DataFrame
display(X.head())
display(X.tail())

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300
77535,15600.0,9.742,72300,0.585062,9,2,42300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [20]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [21]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [22]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model

logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data

lr_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [24]:
# Make a prediction using the testing data

# Generate training predictions for later use
training_predictions = logistic_regression_model.predict(X_train)

# Generate testing predictions
testing_predictions = logistic_regression_model.predict(X_test)

testing_predictions


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
# Print the balanced_accuracy score of the model

balanced_accuracy_score(y_test, testing_predictions)

0.9520479254722232

In [26]:
# Generate a confusion matrix for the traning data 

training_matrix = confusion_matrix(y_train, training_predictions)

# Print the confusion matrix for the training data
print(training_matrix)

[[55994   277]
 [  181  1700]]


In [27]:
# Create a confusioni matrix for the testing data

test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[18663   102]
 [   56   563]]


In [28]:
# Print the classification report for the model

training_report = classification_report(y_train, training_predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56271
           1       0.86      0.90      0.88      1881

    accuracy                           0.99     58152
   macro avg       0.93      0.95      0.94     58152
weighted avg       0.99      0.99      0.99     58152



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model did an outstanding job on predicting the healthy loans in the training data, it predicted with perfect precision and recall.   It did a quite good job of predicting the risky loans, but did nowhere near as well either in terms of precision or recall, with its worst score, .86, on precision for high risk loans.  The model did about 4 percent better at  recall for high risk loans than at precision for those loan predictions.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [30]:
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model

ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model

X_resampled, y_resampled = ros.fit_resample(X, y)

In [32]:
# Count the distinct values of the resampled labels data
y_resampled.value_counts()

0    75036
1    75036
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [54]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model

logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data

resampled_model = logistic_regression_model.fit(X_resampled, y_resampled)

# Make training predictions

resampled_predictions = logistic_regression_model.predict(X_resampled)

print(len(resampled_predictions))
resampled_predictions


150072


array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [53]:
# Again Split the resampled data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, random_state=1)

# Create and fit the logistic regression model on the resampled training data
logistic_regression_model = LogisticRegression(random_state=1)
resampled_model = logistic_regression_model.fit(X_train, y_train)

# Make predictions on the resampled testing set
resampled_predictions = resampled_model.predict(X_test)

print(len(resampled_predictions))
print(len(y_test))


37518
37518


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [50]:
# Print the balanced_accuracy score of the model 

balanced_accuracy_score(y_test, resampled_predictions)

0.9950160462226244

In [56]:
# Generate a confusion matrix for the resampled model

resampled_testing_matrix = confusion_matrix(y_test, resampled_predictions)

# Print the confusion matrix for the training data
print(resampled_testing_matrix)

ValueError: Found input variables with inconsistent numbers of samples: [37518, 150072]

In [ ]:
# Print the classification report for the model
# YOUR CODE HERE!

### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** YOUR ANSWER HERE!